<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/Chapter4_%EC%8B%A4%EC%8A%B5_3_(10%EC%9D%BC%EC%B0%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#github와 colab 연동
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


주어진 학습용 데이터를 활용하여 해당 사람이 연소득이 5만 달러가 넘는지 안 넘는지 예측 모형을 만든 후, 이를 평가용 데이터에 적용하여 얻은 연소득 5만 달러 초과 여부(초과인 경우를 1로 한다)를 다음과 같은 csv 파일로 생성하시오. (제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점.)

In [ ]:
import pandas as pd
import numpy as np

X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/census_X_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/census_X_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/census_y_train.csv')

In [ ]:
#데이터 살펴보기
print(X_train.shape)
print(X_train.info())
#-> 결측치 없음
#-> 문자열 변수인 몇가지 칼럼만 인코딩 작업을 하면됨.

(30162, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB
None


In [ ]:
print(X_train.describe())
#-> capitial_gain과 capital_loss가 오른쪽으로 치우쳐져 있음을 알 수 있음.

                age  education_num  capital_gain  capital_loss  hours_per_week
count  30162.000000   30162.000000  30162.000000  30162.000000    30162.000000
mean      38.437902      10.121312   1092.007858     88.372489       40.931238
std       13.134665       2.549995   7406.346497    404.298370       11.979984
min       17.000000       1.000000      0.000000      0.000000        1.000000
25%       28.000000       9.000000      0.000000      0.000000       40.000000
50%       37.000000      10.000000      0.000000      0.000000       40.000000
75%       47.000000      13.000000      0.000000      0.000000       45.000000
max       90.000000      16.000000  99999.000000   4356.000000       99.000000


In [ ]:
#위의 두 변수를 5 백분위수 단위로 쪼개서 자세히 살펴보기
print(X_train['capital_gain'].quantile([q/20 for q in range(15,21)]))
#capital gain 변수는 전체 10% 이내만 0보다 큰 값을 가짐.

print(X_train['capital_loss'].quantile([q/20 for q in range(15,21)]))
#capital loss 변수는 전체 5% 이내만 0보다 큰 값을 가짐.

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64
0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64


In [ ]:
#범주형 변수로 변환하여 새로운 파생변수 생성
#초도 모델링 이후 변수 중요도를 통해 기존 변수와 파생변수 중 어떤 변수가 더 중요하게 쓰이는지 확인해보기

#where 함수를 이용해서 파생변수 생성
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0,1,0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0,1,0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0,1,0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0,1,0)

In [ ]:
#종속변수 값별로 독립변수 값의 차이가 있는지 탐색하기 위해 변수 종류별로 나눠줌.
COL_DEL = []
COL_NUM = ['age','education_num','hours_per_week','capital_gain','capital_loss']
COL_CAT = ['workclass','marital_status','occupation','relationship','race','sex','native_country','capital_gain_yn','capital_loss_yn']
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)

for _col in COL_NUM:
    print('-'*80)
    print(_col)
    print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

#age, education_num. hours_per_week는 target이 1일 때가 0일 때보다 크다.
#capital_gain과 capital_loss는 종속변수 값에 따른 평균 값의 차이가 크다.
#=> 수치형 변수 모두 종속변수를 예측하는데 사용할만 하다.

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  

In [ ]:
#범주형 변수(COL_CAT에 포함된 열들)를 기반으로 주어진 데이터프레임 train_df을 그룹화
#각 범주에 대한 평균 목표 변수(COL_Y) 값을 계산, 그 값을 내림차순으로 정렬하여 출력하는 작업
for _col in COL_CAT:
    #as_index=False는 그룹화한 열을 인덱스로 사용하지 않음을 나타냄.
    print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False),end='\n\n')

#=> 어떤 범주가 목표 변수에 대해 가장 높은 평균 값을 갖는지 확인하는 데 도움이 됨.
#=> 학습 데이터에서 종속변수가 1인 비율은 약 0.249임.

          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   Transport-moving  0.202926
0        Adm-clerical  0.133835
6   Machine-op-inspct  0.124619
4     Farming-fishing  0.116279
1        Armed-Forces  0.111111
5   Handlers-cleaners  0.061481
7       Other-service  0.041096
8     Priv-house-serv  0.00699

native country 변수의 경우 41개의 고유한 값을 가짐.

한 변수가 가질 수 있는 고유한 값의 개수 = 카디널리티

카디널리티가 높은 변수는 모델 성능에 악영향임.



In [ ]:
#범주형 변수들에 대해 레이블 인코딩 수행
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])
for _col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[_col])
    X_train[_col] = le.transform(X_train[_col])
    X_test[_col] = le.transform(X_test[_col])

In [ ]:
#데이터 모형 구축
from sklearn.model_selection import train_test_split
#stratify는 분할된 훈련 데이터와 검증 데이터에서 각 클래스의 비율이 원본 데이터와 비슷하게 유지하기 위해 사용
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [ ]:
#데이터 학습 진행
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])

In [ ]:
#학습 모델 생성하기

#1. Random Forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

#2. XGBoost
from xgboost import XGBClassifier
model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

#XGBoost 학습 시 eval_set을 사용하여 충분히 학습을 시켜야지 과적합을 방지할 수 있음.
model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(X_tr, y_tr.values.ravel(), early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

[0]	validation_0-auc:0.89925
[10]	validation_0-auc:0.90387


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20]	validation_0-auc:0.90294
[30]	validation_0-auc:0.90285
[40]	validation_0-auc:0.90389
[50]	validation_0-auc:0.90465
[60]	validation_0-auc:0.90444
[70]	validation_0-auc:0.90452
[80]	validation_0-auc:0.90429
[90]	validation_0-auc:0.90379
[100]	validation_0-auc:0.90333
[102]	validation_0-auc:0.90325


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
#ROC-AUC 값을 이용하여 초도 모델의 성능 확인
from sklearn.metrics import roc_auc_score
y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:,1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:,1])

print(score_rf)
print(score_xgb1)
#XGBoost 모델의 성능이 더 좋음.

0.8938120098434399
0.9043955762533412


In [ ]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_estimators':[50,100,200],
    'max_depth':[5,10,15],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4]
}

rf_cv = GridSearchCV(estimator=model_rf, param_grid=grid_params, cv=5)
rf_cv.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [ ]:
print(pd.DataFrame(rf_cv.cv_results_).head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.823439      0.256333         0.062742        0.032156   
1       0.951090      0.217432         0.056866        0.000818   
2       1.868101      0.242364         0.117405        0.011070   
3       0.427182      0.007473         0.029704        0.000779   
4       0.849696      0.010044         0.056906        0.000757   

  param_max_depth param_min_samples_leaf param_min_samples_split  \
0               5                      1                       2   
1               5                      1                       2   
2               5                      1                       2   
3               5                      1                       5   
4               5                      1                       5   

  param_n_estimators                                             params  \
0                 50  {'max_depth': 5, 'min_samples_leaf': 1, 'min_s...   
1                100  {'max_depth': 5,

In [ ]:
#best_params_ 속성을 사용하면 그리드 서치 중 가장 좋은 성능을 보였던 estimator를 불러올 수 있음.
print(rf_cv.best_params_)

{'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [ ]:
#Random Forest 모델 성능 향상시키기
model_rf2 = RandomForestClassifier(n_estimators=50, #부스팅 iteration 수, 일반적으로 100~1000 사이의 값을 가짐
                                   max_depth=15, #트리 깊이의 최댓값, 일반적으로 3~10 사이의 값을 가짐
                                   min_samples_leaf=1,
                                   min_samples_split=5)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:,1])
print(score_rf2)
#여기서 max_depth를 가장 주의 깊게 봐야힘.
#why? 해당 값이 너무 작으면 과소적합, 너무 크면 과대적합의 위험이 있기 때문임.

0.914026529570694


In [ ]:
#XGBoost 모델 성능 향상시키기 (GridSearchCV 함수를 사용하여~!)
grip_params = {'max_depth':[3,5,7,10], #트리 깊이의 최댓
               'min_child_weight':[1,2], #자식 노드에 필요한 가중치 합의 최솟값
               'colsample_bytree':[0.6, 0.8], #각 트리에 사용할 변수 비율
               'subsample':[0.6,0.8]} #각 트리에 사용할 데이터 샘플 비율
xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=grid_params, cv=5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:30:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:30:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:30:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:30:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [ ]:
print(xgb_cv.best_params_)

{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [ ]:
params = {'colsample_bytree':0.6,
          'max_depth':7,
          'min_child_weight':1,
          'subsample':0.8}
model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

print(model_xgb3.best_score)

[0]	validation_0-auc:0.88747
[10]	validation_0-auc:0.90875
[20]	validation_0-auc:0.91077


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[30]	validation_0-auc:0.91148
[40]	validation_0-auc:0.91150
[50]	validation_0-auc:0.91123
[60]	validation_0-auc:0.91089
[70]	validation_0-auc:0.91149
[80]	validation_0-auc:0.91172
[90]	validation_0-auc:0.91259
[100]	validation_0-auc:0.91291
[110]	validation_0-auc:0.91319
[120]	validation_0-auc:0.91337
[130]	validation_0-auc:0.91347
[140]	validation_0-auc:0.91366
[150]	validation_0-auc:0.91368
[160]	validation_0-auc:0.91339
[170]	validation_0-auc:0.91298
[180]	validation_0-auc:0.91282
[190]	validation_0-auc:0.91280
[198]	validation_0-auc:0.91274
0.9137773599835473


In [ ]:
pred = model_xgb3.predict_proba(X_test)[:,1]
pd.DataFrame({'index':X_test.index, 'target':pred}).to_csv('./yemoonsaBigdata/res/003000000.csv', index=False)